# Chương 2: Làm việc với dữ liệu văn bản

Cài đặt thư viện
- Pytorch: https://pytorch.org/get-started/locally/
- `pip install tiktoken`

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.6.0+cu124
tiktoken version: 0.9.0


<img src="https://images.viblo.asia/ca53ac05-df27-4ae9-86f7-f8441fd5c8b5.png">

## 2.1 Word embeddings

Các mô hình ngôn ngữ như LLMs không thể xử lý trực tiếp dữ liệu văn bản, thay vào đó cần phải **Toán học hóa** chúng bằng cách chuyển đổi thành các vector (dạng số học) để mô hình có thể hiểu và xử lý thông tin.

> Mọi thứ đều là số - **Pythagoras**

Khái niệm chuyển đổi dữ diệu thành các vector gọi là **embeddings**. Không chỉ có dữ liệu văn bản, mà các kiểu dữ liệu khác như âm thanh, video cũng được chuyển sang dạng vector trước khi xử lý.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

Một trong những thuật toán phổ biến dùng trong việc chuyển đổi văn bản được nhóm nghiên cứu của Google công bố vào năm 2013 có tên là **Word2Vec**.

**Word2Vec** dự đoán ngữ cảnh của một từ dựa trên từ mục tiêu hoặc ngược lại. Ý tưởng chính đằng sau Word2Vec là các từ xuất hiện trong những ngữ cảnh tương tự có xu hướng mang ý nghĩa giống nhau.

Ví dụ ta có các câu sau trong tập dữ liệu huấn luyện:

1. "Nhà vua trị vì đất nước của ông."
2. "Nữ hoàng cai trị vương quốc của bà."

**Word2Vec** học được rằng "vua" và "nữ hoàng" thường xuất hiện trong ngữ cảnh tương tự. Kiểm tra khoảng cách giữa 2 vector cũng sẽ thấy rất gần nhau.

Do đó, khi được chiếu vào không gian hai chiều để trực quan hóa, các từ có ý nghĩa tương tự sẽ được nhóm lại với nhau, như minh họa trong hình dưới đây.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

Trong thực tiễn, các từ sẽ được biểu diễn ở số chiều lớn hơn nhiều so với hình học không gian cổ điển. 2 đến 3 chiều là không đủ để biểu diễn, phân biệt hàm trăm nghìn, hàng triệu hoặc từ trong các bộ dữ liệu văn bản.

| Phiên bản | Số chiều vector embeddings |
| -------- | -------- |
| GPT-2     | 768     | 
| GPT-2 Large     | 1280     | 
| GPT-2 XL    | 1600   | 
| GPT-3 (175B)     | 12288  | 

Tuy nhiên, các LLMs như **GPT** sẽ không dùng **Word2Vec** mà có cách riêng để tạo **embledding**. Do phương pháp này bộc lộ một số hạn chế sau :
- Nếu một từ có nhiều nghĩa khác nhau, Word2Vec vẫn chỉ tạo một vector duy nhất.
- Chưa xử lý tốt các văn bản dài: Word2Vec chỉ dựa vào ngữ cảnh gần của từ mà không xem xét ngữ cảnh lớn hơn

## 2.2 Tokenizing text

**Tokenizing text** (hay tokenization) là quá trình chuyển đổi một chuỗi văn bản thành các đơn vị nhỏ hơn, gọi là **tokens**. Các tokens có thể là một từ, cụm từ, hoặc một ký hiệu, tùy thuộc vào ngữ cảnh và mục đích của việc xử lý văn bản.

Dưới đây là hình minh họa các bước xử lý để tạo ra **token embeddings** của họ nhà GPT.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp">

- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict): File văn bản của một câu chuyện ngắn

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [3]:
# Mở file và đọc nội dung
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Xây dựng các phiên bản tách văn bản thành các token từ đơn giản đến phức tạp

In [4]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text) # Tách chuỗi theo khoảng trắng

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [5]:
result = re.split(r'([,.]|\s)', text) # Tách chuỗi theo dấu phẩy, dấu chấm và khoảng trắng

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [6]:
# Bỏ bớt các phần tử rỗng hoặc chỉ có khoảng trắng
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [7]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text) # Tách chuỗi theo dấu câu và khoảng trắng
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [8]:
# Thực hiện trên file the-verdict.txt
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- Tính tổng số token

In [9]:
print(len(preprocessed))

4690


## 2.3 Chuyển đổi tokens sang token IDs

Sau khi đã tách văn bản thành các **tokens**, bước tiếp theo sẽ là mã hóa các tokens thành các token IDs. Mỗi token sẽ ứng với một giá trị khác nhau được định nghĩa trong tệp từ vựng (**Vocabulary**).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp">

- Từ các tokens, xây dựng tệp từ vựng 

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)} # Mỗi token ứng với một số nguyên theo thứ tự tăng dần

- Xem 50 từ đầu

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- Hình minh hoạ với 1 tệp từ vựng nhỏ

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123">

- Các logic đã xây dựng viết vào một class

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- Hàm `encode`: Chuyển token -> tokenId
- The `decode`: Chuyển tokenId -> token

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [15]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [16]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 Thêm ký tự đặc biệt

- Các từ không có trong từ điển thì thay bằng `<|unk|>`
- `|endoftext|` để báo hiệu kết thúc văn bản

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- Minh hoạ việc sử dụng `<|endoftext|>`

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

In [18]:
tokenizer = SimpleTokenizerV1(vocab)
text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- Ở trên lỗi vì từ "Hello" trong có trong `vocab`
- Để xử lý TH này, thêm ký tự đặc biệt `<|unk|>` vào `vocab`. Khi đó "Hello" => `<|unk|>`

In [19]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [20]:
len(vocab.items())

1132

In [21]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- Sửa hàm `encode` để xử lý `<unk>`

In [23]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

Let's try to tokenize text with the modified tokenizer:

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [26]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [27]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 BytePair encoding

Phương pháp tạo ra tệp từ vựng (Vocabulary) như trên sẽ bộc lộ hạn chế khi làm việc với số lượng từ rất lớn. Trong thực tiễn, các mô hình GPT hay Llama sử dụng một thuật toán phức tạp hơn gọi là **Byte pair encoding (BPE)**.

**BPE** hoạt động dựa trên việc hợp nhất các cặp ký tự hoặc âm tiết phổ biến trong văn bản để tạo thành tệp từ vựng. Một ví dụ đơn giản như sau:

### Bước 1:
Một văn bản dài chỉ được tạo thành bởi 5 từ sau `"hug pug pun bun hugs"`

### Bước 2:
Từ vựng cơ sở là các chữ cái tạo nên văn bản, khi này **Tệp từ vựng** là `["b", "g", "h", "n", "p", "s", "u"]`

### Bước 3:
Đếm tần số xuất hiện của các từ `("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)`

### Bước 4:
Lại tách từng từ thành các ký tự như sau:

`("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)`

### Bước 5:
Cặp **ug** có tần suất nhiều nhất (20 lần). Ta tiến hành hợp nhất `("u", "g") -> "ug"` 

**Tệp từ vựng** mới là: `["b", "g", "h", "n", "p", "s", "u", "ug"]`

**Danh sách các tokens**: `("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)`

### Bước 6:

Lại xét cặp **un** (16 lần) nhiều nhất. Tiến hành hợp nhất `("u", "n") -> "un"`

**Tệp từ vựng** mới là: `["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]`

**Danh sách các tokens**: `("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)`

Tiếp tục làm vậy cho đến khi chúng ta chạm đến kích thước tệp từ vựng mong muốn.


> ChatGPT đang dùng tệp từ vựng gồm có **50257** tokens (https://openaipublic.blob.core.windows.net/gpt-2/encodings/main/encoder.json)


In [25]:
# pip install tiktoken

In [29]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [30]:
tokenizer = tiktoken.get_encoding("gpt2")

In [31]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [33]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE không dùng `<|unk|>` mà phân chia các từ chưa biết thành các từ nhỏ hơn và ký tự.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="600px">

## 2.6 Cửa sổ trượt (sliding window)

- Như đã biết, LLMs dựa vào các từ trước đó để sinh ra từ tiếp theo. Do đó, cần sắp xếp dữ liệu sao cho mô hình "không nhìn thấy" được các từ kế tiếp.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="600px">

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text)) # Số lượng token

5145


- Đối với mỗi đoạn văn bản, chúng ta cần 2 giá trị là `input` và `target`
- Vì chúng ta muốn mô hình dự đoán từ tiếp theo, `target` dịch chuyển sang phải một vị trí so với `input`

In [31]:
enc_sample = enc_text[50:]

In [32]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [33]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


Các mô hình ngôn ngữ lớn thường có giới hạn về kích thước ngữ cảnh (**context window**), tức là số lượng token mà mô hình có thể xử lý cùng một lúc.

**Sliding Window** là một kỹ thuật cho phép chia nhỏ các chuỗi văn bản dài thành các đoạn ngắn hơn và xử lý chúng một cách tuần tự. Các đoạn ngắn hơn, mỗi đoạn có độ dài nhất định (window size).

Hai đặc tính của **sliding window**:
- **Context length**: Độ dài của mỗi đoạn sau khi được chia nhỏ.
- **Stride**: Thông số thể hiện sự xê dịch giữa các đoạn với nhau.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- Create dataset and dataloader that extract chunks from the input text dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize toàn bộ văn bản
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Sử dụng sliding window với độ dài max_length là tham số truyền vào
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [37]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # thư viện tiktoken của OpenAI, dùng kỹ thuật BPE
    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [38]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [39]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- Ảnh minh hoạ với 2 TH `stride = 1` và `stride = 4`

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

Như ảnh trên thì thấy dùng stride = 4 sẽ hạn chế việc trùng lặp nhiều

In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 Tạo Token embeddings

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="600px">

In [ ]:
# Ví dụ có 4 token ids
input_ids = torch.tensor([2, 3, 5, 1])

### Ma trận embedding

Ta có ma trận embedding $W \in \mathbb{R}^{V \times d}$, trong đó:
- **V** (số  cột):  Kích thước từ vựng (vocabulary size)
- **d** (số hàng): Số chiều của embeddings, ví dụ: `d=768` trong GPT-2.

token Id `X` sẽ được tham chiếu với hàng thứ `X + 1` trong ma trận embedding.

Vậy câu hỏi đặt ra là lấy ra cái ma trận embedding này ở đâu ? Xin thưa là **ma trận embedding** được tạo ra bằng cách huấn luyện sử dụng các phương pháp khác như `one-hot encoding` hay mạng nơ-ron ... Chi tiết có lẽ chúng ta sẽ gặp nhau ở một bài viết khác nói riêng về chủ đề này 😄


In [ ]:
vocab_size = 6 # V = 6
output_dim =  3 # d = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim) # Ma trận embeddings

- Kích thước ma trận là 6x3

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [ ]:
print(embedding_layer(torch.tensor([3]))) # Chuyển tokenId = 3 thành token embedding

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [ ]:
print(embedding_layer(input_ids)) # Chuyển 4 token ids ban đầu thành 4 token embeddings

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


Minh hoạ:
- Ví dụ ta có tokenId 5. Ta tìm đến hàng thứ 6 của ma trận embedding => Đó là giá trị token embeddings của tokenId = 5

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

## 2.8 Vị trí của từ trong câu (word positions)

Với đầu ra **token embeddings** ở phần trên, chúng ta đã có thể đưa vào mô hình để chạy.

Tuy nhiên, chưa có cách nào phân biệt hai từ giống nhau nằm ở các vị trí khác nhau. Điều này ảnh hưởng rất lớn đến việc mô hình hiểu được ngữ nghĩa, cấu trúc trong câu vì thứ tự từ trong câu thay đổi có thể thay đổi ý nghĩa của toàn bộ câu.

Ví dụ, câu `Con mèo đuổi theo con chuột` có ý nghĩa khác với `Con chuột đuổi theo con mèo`.

Giải pháp ở đây là chúng ta sẽ thêm một **position embeddings** cộng với **token embeddings** để ra kết quả cuối cùng là **input embeddings**.

Hiện này có 2 cách phổ biến để tính toán *position embeddings* là **Positional Encoding (PE)** và **Learnable Position Embeddings (LPE)**

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="600px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

Hiện này có 2 cách phổ biến để tính toán *position embeddings* là **Positional Encoding (PE)** và **Learnable Position Embeddings (LPE)**

### Positional Encoding (PE)

**Positional Encoding** được sử dụng trong **Transformer** và các mô hình đời đầu của GPT như GPT-2. Phương pháp này sử dùng 2 hàm lượng giác **sin và cos** để tính toán các vector vị trí.

$$
\begin{aligned}
PE_{(pos, 2i)} &= \sin\left(\frac{pos}{10000^{2i/d}}\right) \\
\\
PE_{(pos, 2i+1)} &= \cos\left(\frac{pos}{10000^{2i/d}}\right)
\end{aligned}
$$

hay

$$
PE_{(pos, k)} =
\begin{cases} 
 \sin\left(\frac{pos}{10000^{2i/d}}\right) & \text{Nếu } k = 2i \text{ với } 0 \leq i < \frac{d}{2}  \\
\cos\left(\frac{pos}{10000^{2i/d}}\right) & \text{Nếu } k = 2i + 1 
\text{ với } 0 \leq i < \frac{d}{2}
\end{cases}
$$

- **pos** là vị trí của từ trong câu (0 <= pos < L / 2 với **L** là độ dài của câu)
- **d** là số chiều của embedding
- **i** dùng để tính vị trí phần tử trong mỗi hàng

|  Từ | pos | k = 0, i = 0 |k = 1,  i = 0 |k = 2,  i = 1 |k = 3,  i = 1 |
| -------- | -------- | -------- | -------- | -------- | -------- |
| I     | 0   | $PE_{(0, 0)}$ = sin(0) = 0     |  $PE_{(0, 1)}$ = cos(0) = 1|  $PE_{(0, 2)}$ = sin(0) = 0|  $PE_{(0, 3)}$ = cos(0) = 1|
| am     | 1     |  $PE_{(1, 0)}$ = sin(1) = 0.84     | $PE_{(1, 1)}$ = cos(1) = 0.54| $PE_{(1, 2)}$ = sin(1/100) = 0.0099| $PE_{(1, 3)}$ = cos(1/100) = 0.99995|
| a     | 2     |  $PE_{(2, 0)}$ = sin(2) = 0.91    | $PE_{(2, 1)}$ = cos(2) = -0.42 | $PE_{(2, 2)}$ = sin(2/100) = 0.019 | $PE_{(2, 3)}$ = cos(2/100) = 0.9998 |
| Robot    | 3     |  $PE_{(3, 0)}$ = sin(3) = 0.14    |  $PE_{(3, 1)}$ = cos(3) = -0.99 |  $PE_{(3, 2)}$ = sin(3/100) = 0.029 |  $PE_{(3, 3)}$ = cos(3/100) = 0.99 |

```python
# postion embeddings
tensor([[ 0, 1, 0, 1],
        [ 0.84, 0.54, 0.0099, 0.99995],
        [ 0.91, -0.42, 0.019, 0.9998],
        [ 0.14, -0.99, 0.029, 0.99]
])
```

### Learnable Position Embeddings (LPE)

Giống như phương pháp tạo **token embeddings**, LPE cung cấp một ma trận vị trí kích thước **(L, d)**

$$PE = W_{pos}$$
- L là độ dài của chuỗi
- d số chiều của embeddings

Vị trí nào sẽ ứng với dòng đó trong ma trận khi chuyển đổi.

## So sánh PE và LPE

| | Positional Encoding (PE) | Learnable Position Embeddings (LPE) |
| -------- | -------- | -------- |
| Cách tính toán     | Dùng hàm lượng giác     | Sử dụng ma trận |
| Huấn luyện được    | Không     | Có |
| Bộ nhớ    | Ít tốn bộ nhớ     | Tốn nhiều bộ nhớ hơn |
| Có thể tùy chỉnh    | Không     | Có |

Ví dụ với Python

In [46]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
# Dùng sliding window để chia nhỏ
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [48]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape) # Kích thước của ma trận embeddings

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2 dùng Learnable Position Embeddings (LPE) nên cần tạo 1 ma trận `pos_embedding_layer`

In [51]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
# print(pos_embedding_layer.weight)

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])


- Tính `input_embeddings = token_embeddings + pos_embeddings`


In [53]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)

torch.Size([8, 4, 256])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="600px">

# Summary and takeaways

[./dataloader.ipynb](./dataloader.ipynb) chứa code hoàn chỉnh từ bài này

Đáp án bài tập trong sách [./exercise-solutions.ipynb](./exercise-solutions.ipynb)

Sâu hơn về [Byte Pair Encoding (BPE) Tokenizer From Scratch](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) coi ở đây.